# Installer les dépendances

Attention: Ces instructions ne fonctionnent que sur un système Unix. Si vous utilisez Windows, vous pouvez utiliser WSL (Windows Subsystem for Linux) ou installer les dépendances manuellement.

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Utiliser des requêtes HTTP pour récupérer le contenu d'une page web

In [5]:
SITE = "https://books.toscrape.com/"

response = requests.get(SITE)

Vérifions le code HTTP que retourne la requête:

In [6]:
print(response.status_code)

200


Le code HTTP 200 signifie que la requête a réussi.

On peut maintenant récupérer le contenu de la page à partir de la réponse:

In [7]:
print(response.text[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

# Récolter des données


In [8]:
# Analyser le contenu HTML
soup = BeautifulSoup(response.text, "html.parser")

# Obtenir toutes les balises <article> de la page
articles = soup.find_all("article")

print(f"Nombre d'articles : {len(articles)}")
print(articles[0].prettify())

Nombre d'articles : 20
<article class="product_pod">
 <div class="image_container">
  <a href="catalogue/a-light-in-the-attic_1000/index.html">
   <img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/>
  </a>
 </div>
 <p class="star-rating Three">
  <i class="icon-star">
  </i>
  <i class="icon-star">
  </i>
  <i class="icon-star">
  </i>
  <i class="icon-star">
  </i>
  <i class="icon-star">
  </i>
 </p>
 <h3>
  <a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">
   A Light in the ...
  </a>
 </h3>
 <div class="product_price">
  <p class="price_color">
   Â£51.77
  </p>
  <p class="instock availability">
   <i class="icon-ok">
   </i>
   In stock
  </p>
  <form>
   <button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">
    Add to basket
   </button>
  </form>
 </div>
</article>



# Extraire des données structurées à partir du contenu HTML

In [9]:
def extract_book_info(article):
    # En utilisant find
    title = article.find("h3").find("a")["title"]
    price_string = article.find("p", class_="price_color").text
    price_pounds = float(price_string.replace("Â£", ""))

    # En utilisant un selecteur CSS
    image = article.select_one("img")["src"]
    # Créer un URL absolu
    image_url = f"{SITE}{image}"

    book_url = article.select_one("h3 a").get("href")
    book_url = f"{SITE}{book_url}"

    stars = article.find("p", class_="star-rating")["class"][1]

    in_stock = (
        article.find("p", class_="instock availability").text.strip() == "In stock"
    )
    return {
        "title": title,
        "price": price_pounds,
        "image_url": image_url,
        "in_stock": in_stock,
        "stars": stars,
        "book_url": book_url,
    }

In [10]:
for article in articles:
    print(extract_book_info(article))

{'title': 'A Light in the Attic', 'price': 51.77, 'image_url': 'https://books.toscrape.com/media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg', 'in_stock': True, 'stars': 'Three', 'book_url': 'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'}
{'title': 'Tipping the Velvet', 'price': 53.74, 'image_url': 'https://books.toscrape.com/media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg', 'in_stock': True, 'stars': 'One', 'book_url': 'https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html'}
{'title': 'Soumission', 'price': 50.1, 'image_url': 'https://books.toscrape.com/media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg', 'in_stock': True, 'stars': 'One', 'book_url': 'https://books.toscrape.com/catalogue/soumission_998/index.html'}
{'title': 'Sharp Objects', 'price': 47.82, 'image_url': 'https://books.toscrape.com/media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg', 'in_stock': True, 'stars': 'Four', 'book_url': 'https://books.toscrape.

On va transformer ces données en un DataFrame pandas pour rendre les données plus faciles à manipuler

In [11]:
df = pd.DataFrame([extract_book_info(article) for article in articles])
print(df)

                                                title  price  \
0                                A Light in the Attic  51.77   
1                                  Tipping the Velvet  53.74   
2                                          Soumission  50.10   
3                                       Sharp Objects  47.82   
4               Sapiens: A Brief History of Humankind  54.23   
5                                     The Requiem Red  22.65   
6   The Dirty Little Secrets of Getting Your Dream...  33.34   
7   The Coming Woman: A Novel Based on the Life of...  17.93   
8   The Boys in the Boat: Nine Americans and Their...  22.60   
9                                     The Black Maria  52.15   
10     Starving Hearts (Triangular Trade Trilogy, #1)  13.99   
11                              Shakespeare's Sonnets  20.66   
12                                        Set Me Free  17.46   
13  Scott Pilgrim's Precious Little Life (Scott Pi...  52.29   
14                          Rip it Up an

Okay ! C'est super, mais il y a plus d'une page à scraper.

# Pagination 1: Méthode linéaire

On commence par modulariser le code un poil: on va créer une fonction qui récupère les données d'une page

In [12]:
def get_page_books(page_idx: int) -> list[dict]:
    response = requests.get(f"{SITE}catalogue/page-{page_idx}.html")
    soup = BeautifulSoup(response.text, "html.parser")
    return [extract_book_info(article) for article in soup.find_all("article")]

On va commencer par utiliser une méthode simple pour gérer la pagination: on va boucler sur les pages et récupérer les données de chaque page. Ca demande de savoir à l'avance le nombre de pages à scraper. Ici, on sait que le site a 50 pages.

In [13]:
all_books = pd.DataFrame()
# Méthode linéaire
for page_idx in range(1, 51):
    books = get_page_books(page_idx)
    print(f"Page {page_idx} : {len(books)} books")
    all_books = pd.concat([all_books, pd.DataFrame(books)])

print(all_books)

Page 1 : 20 books
Page 2 : 20 books
Page 3 : 20 books
Page 4 : 20 books
Page 5 : 20 books
Page 6 : 20 books
Page 7 : 20 books
Page 8 : 20 books
Page 9 : 20 books
Page 10 : 20 books
Page 11 : 20 books
Page 12 : 20 books
Page 13 : 20 books
Page 14 : 20 books
Page 15 : 20 books
Page 16 : 20 books
Page 17 : 20 books
Page 18 : 20 books
Page 19 : 20 books
Page 20 : 20 books
Page 21 : 20 books
Page 22 : 20 books
Page 23 : 20 books
Page 24 : 20 books
Page 25 : 20 books
Page 26 : 20 books
Page 27 : 20 books
Page 28 : 20 books
Page 29 : 20 books
Page 30 : 20 books
Page 31 : 20 books
Page 32 : 20 books
Page 33 : 20 books
Page 34 : 20 books
Page 35 : 20 books
Page 36 : 20 books
Page 37 : 20 books
Page 38 : 20 books
Page 39 : 20 books
Page 40 : 20 books
Page 41 : 20 books
Page 42 : 20 books
Page 43 : 20 books
Page 44 : 20 books
Page 45 : 20 books
Page 46 : 20 books
Page 47 : 20 books
Page 48 : 20 books
Page 49 : 20 books
Page 50 : 20 books
                                                title  pric

On peut maintenant exporter les données dans un fichier CSV

In [14]:
all_books.to_csv("data/all_books.csv", index=False)

Et maintenant, on a un jeu de données avec lequel on peut faire des analyses !

# Pagination 2: Méthode récursive

La méthode linéaire est simple, mais elle n'est pas optimale puisqu'elle demande de savoir à l'avance le nombre de pages à scraper, ce qui n'est pas toujours évident. On peut faire mieux en utilisant une méthode récursive.

Par exemple ici, on va récupérer tous les livres par catégories. On ne sait pas à l'avance le nombre de pages pour chaque catégorie. On pourrait aller les compter à la main, mais ce serait un peu long.

Avec une méthode récursive, on va pouvoir aller récupérer la prochaine page tant qu'il y a des données à récupérer.

On va commencer par récupérer la homepage, et récupérer les liens vers chaque catégorie.

In [15]:
homepage = requests.get(SITE)
soup = BeautifulSoup(homepage.text, "html.parser")

# Comment trouve-t-on ce sélecteur CSS ?
categories_html = soup.select("ul.nav-list li ul li a")

categories_urls = []
for category_html in categories_html:
    category_url = f"{SITE}{category_html.get('href')}"
    categories_urls.append(category_url)
    print(category_url)

https://books.toscrape.com/catalogue/category/books/travel_2/index.html
https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
https://books.toscrape.com/catalogue/category/books/classics_6/index.html
https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html
https://books.toscrape.com/catalogue/category/books/romance_8/index.html
https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html
https://books.toscrape.com/catalogue/category/books/fiction_10/index.html
https://books.toscrape.com/catalogue/category/books/childrens_11/index.html
https://books.toscrape.com/catalogue/category/books/religion_12/index.html
https://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html
https://books.toscrape.com/catalogue/category/books/music_14/index.html
https://books.

Et voilà, on a nos catégories ! On peut maintenant récupérer les livres de chaque catégorie.

In [16]:
def get_next_page(soup: BeautifulSoup, category_url: str) -> str | None:
    next_page = soup.select_one("li.next a")
    if next_page:
        # Petite subtilité: le lien est un lien relatif au 3ème niveau de l'URL (bizarre),
        # on doit donc le transformer en lien absolu:
        # Enlever le index.html de l'URL actuelle
        category_url = category_url.replace("index.html", "")
        # Puis ajouter le lien relatif au 3ème niveau
        return f"{category_url}{next_page.get('href')}"
    return None

In [17]:
all_books = pd.DataFrame()

for category_url in categories_urls:
    print(f"Récupération des livres de la catégorie {category_url}")

    # On récupère la première page de la catégorie
    response = requests.get(category_url)
    soup = BeautifulSoup(response.text, "html.parser")
    books = [extract_book_info(article) for article in soup.find_all("article")]

    # On récupère la deuxième page de la catégorie s'il y en a une
    next_page = get_next_page(soup, category_url)

    # Puis on va récupérer les livres de la page suivante tant qu'il y a une page suivante
    while next_page:
        response = requests.get(next_page)
        soup = BeautifulSoup(response.text, "html.parser")
        books.extend(
            [extract_book_info(article) for article in soup.find_all("article")]
        )
        next_page = get_next_page(soup, category_url)

    print(f"Récupéré {len(books)} livres pour la catégorie {category_url}")

    all_books = pd.concat([all_books, pd.DataFrame(books)])

print(all_books)

Récupération des livres de la catégorie https://books.toscrape.com/catalogue/category/books/travel_2/index.html
Récupéré 11 livres pour la catégorie https://books.toscrape.com/catalogue/category/books/travel_2/index.html
Récupération des livres de la catégorie https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
Récupéré 32 livres pour la catégorie https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
Récupération des livres de la catégorie https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
Récupéré 26 livres pour la catégorie https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
Récupération des livres de la catégorie https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
Récupéré 75 livres pour la catégorie https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
Récupération des livres de la catégorie https://books.toscrape.com/catalog

In [18]:
all_books.to_csv("data/all_books_recursive.csv", index=False)

Maintenant on a un jeu de données équivalent, si dans un ordre différent, mais avec une méthode plus adaptable.

Ici le fait de récupérer les données par catégories est un peu artificiel, mais c'est un bon exemple pour montrer comment on peut faire de la pagination récursive.

# More data

Vous aurez peut-être remarqué qu'on a pas récupéré toutes les données disponibles. En effet, on a seulement récupéré jusqu'ici les informations "brèves" des livres. On peut récupérer plus d'informations sur chaque livre en allant sur la page du livre.

On va donc améliorer notre fonction `extract_book_info` pour récupérer plus d'informations sur chaque livre en allant sur la page du livre.

In [19]:
def extract_book_info(article: BeautifulSoup) -> dict:
    # On récupère l'URL du livre et on la rend absolue
    url = article.select_one("div.image_container a").get("href")
    book_url = f"{SITE}{url}"

    # On récupère la page du livre
    response = requests.get(book_url)
    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.select_one("table")

    # On récupère les informations du livre
    return {
        "title": soup.find("h1").text,
        "url": book_url,
        "price": soup.select_one("p.price_color").text,
        "category": soup.select_one("ul.breadcrumb li:nth-child(3) a").text,
        # Sélecteur CSS un peu tricky ! Que fait-il ?
        "description": soup.select_one("div#product_description + p").text,
        "product_type": table.select_one("th:contains('Product Type') + td").text,
        "upc": table.select_one("th:contains('UPC') + td").text,
        "price_ex_tax": table.select_one("th:contains('Price (excl. tax)') + td").text,
        "price_incl_tax": table.select_one(
            "th:contains('Price (incl. tax)') + td"
        ).text,
        "tax": table.select_one("th:contains('Tax') + td").text,
        "availability": table.select_one("th:contains('Availability') + td").text,
        "number_reviews": table.select_one(
            "th:contains('Number of reviews') + td"
        ).text,
    }

In [20]:
more_data_books = pd.DataFrame([extract_book_info(article) for article in articles])
print(more_data_books)

/Users/frieder/Documents/GitHub/web-scraping-workshop/.venv/lib/python3.13/site-packages/soupsieve/css_parser.py:862: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


                                                title  \
0                                A Light in the Attic   
1                                  Tipping the Velvet   
2                                          Soumission   
3                                       Sharp Objects   
4               Sapiens: A Brief History of Humankind   
5                                     The Requiem Red   
6   The Dirty Little Secrets of Getting Your Dream...   
7   The Coming Woman: A Novel Based on the Life of...   
8   The Boys in the Boat: Nine Americans and Their...   
9                                     The Black Maria   
10     Starving Hearts (Triangular Trade Trilogy, #1)   
11                              Shakespeare's Sonnets   
12                                        Set Me Free   
13  Scott Pilgrim's Precious Little Life (Scott Pi...   
14                          Rip it Up and Start Again   
15  Our Band Could Be Your Life: Scenes from the A...   
16                             

In [21]:
more_data_books.to_csv("data/more_data_books.csv", index=False)

# Notions avancées

## Headers

Les headers sont des informations supplémentaires envoyées avec la requête HTTP. Ils permettent de donner des informations supplémentaires sur la requête.

Par exemple, on peut utiliser les headers pour indiquer que la requête est faite par un bot, ou pour indiquer que la requête est faite par un navigateur spécifique: le User-Agent.

Certains sites web utilisent les headers pour détecter les bots et les bloquer. On peut modifier le User-Agent pour masquer notre bot. Ceci est appelé "User-Agent spoofing" ou plus généralement "HTTP header spoofing", et est une technique courante pour éviter d'être détecté comme un bot.

On peut spécifier un header dans la requête avec le paramètre `headers` de la fonction `requests.get`.


In [22]:
firefox_user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
response = requests.get(SITE, headers={"User-Agent": firefox_user_agent})
print(response.text)

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

Beaucoup de ressources gratuites existent pour récupérer des User-Agent valides.

Par exemple, vous pouvez utiliser [https://www.useragents.me/](https://www.useragents.me/) pour récupérer un User-Agent valide.
Vous pouvez même automatiquement les récupérer avec un script Python, et en sélectionner un au hasard. La librairie `fake-useragent` est très pratique pour ça.

In [23]:
!pip install fake-useragent


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [24]:
import fake_useragent

user_agent = fake_useragent.UserAgent().random
print(user_agent)

Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Mobile Safari/537.36


Il y a plein d'autres headers que vous pouvez spécifier, comme le `Referer`, `Accept`, `Accept-Language`, `Accept-Encoding`, `Cookie`, `Host`, `Origin`, `Referer`, `X-Forwarded-For`, etc. Je n'ai pas pour l'heure de recommandations pour les utiliser, mais il y a des cas où ça peut être utile pour construire une requête HTTP plus réaliste.

Vous pouvez trouver la liste complète des headers sur [https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers).

## Cookies

Les cookies sont des informations stockées dans le navigateur de l'utilisateur. Ils permettent de stocker des informations sur l'utilisateur, comme ses préférences, ses favoris, mais aussi sa session d'authentification.

On peut ajouter des cookies à la requête avec le paramètre `cookies` de la fonction `requests.get`.
Un cas pratique est de récupérer les cookies depuis Firefox avec l'extension `Export Cookies` et de les ajouter à la requête.
L'extension permet de récupérer les cookies au format Netscape HTTP Cookie File, que l'on peut directement utiliser avec la fonction `requests.get`.